<a href="https://colab.research.google.com/github/akds2311/Image-classification-/blob/master/vgg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [3]:
!ls "/content/drive/My Drive/Colab Notebooks"

siftf.ipynb  sift.ipynb  sift.py  Untitled0.ipynb


In [0]:
!pip install -q tensorflow

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL
from glob import glob

In [7]:
from keras.models import Model
from keras.layers import Flatten, Dense
from keras.applications import VGG16

Using TensorFlow backend.


In [0]:
IMAGE_SIZE = [224, 224]  # we will keep the image size as (64,64). You can increase the size for better results.

# loading the weights of VGG16 without the top layer. These weights are trained on Imagenet dataset.
conv_base = VGG16(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)  # input_shape = (64,64,3) as required by VGG

# this will exclude the initial layers from training phase as there are already been trained.
for layer in conv_base.layers:
    layer.trainable = False

x = Flatten()(conv_base.output)
#x = Dense(128, activation = 'relu')(x)   # we can add a new fully connected layer but it will increase the execution time.
x = Dense(5, activation = 'softmax')(x)  # adding the output layer with softmax function as this is a multi label classification problem.

model = Model(inputs = conv_base.input, outputs = x)

In [11]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input

training_datagen = ImageDataGenerator(
                                    rescale=1./255,   # all pixel values will be between 0 an 1
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    preprocessing_function=preprocess_input, validation_split=0.1993)


In [0]:
training_dir="/content/drive/My Drive/Colab Notebooks/training"
validation_dir="/content/drive/My Drive/Colab Notebooks/training"
image_files = glob(training_dir + '/*/*.jpg')
# valid_image_files = glob(validation_dir + '/*/*.jpg')

# getting the number of classes i.e. type of fruits
folders = glob(training_dir + '/*')
num_classes = len(folders)


In [17]:
training_generator = training_datagen.flow_from_directory(training_dir, target_size = IMAGE_SIZE, batch_size = 200, class_mode = 'categorical',subset='training')
validation_generator = training_datagen.flow_from_directory(validation_dir, target_size = IMAGE_SIZE, batch_size = 200, class_mode = 'categorical', subset='validation')
training_generator.class_indices

Found 1943 images belonging to 5 classes.
Found 480 images belonging to 5 classes.


{'aircrafts': 0, 'birds': 1, 'cars': 2, 'dogs': 3, 'flowers': 4}

In [0]:
training_images = 1943
validation_images =480


In [20]:
 history = model.fit_generator(training_generator,
                       steps_per_epoch = 100, 
                       epochs = 1,
                       validation_data = validation_generator,
                       validation_steps = 50) 

Epoch 1/1
100/100 [==============================] - 280s 3s/step - loss: 0.0914 - acc: 0.9662 - val_loss: 0.1092 - val_acc: 0.9580


In [21]:
print ('Training Accuracy = ' + str(history.history['acc']))
print ('Validation Accuracy = ' + str(history.history['val_acc']))

Training Accuracy = [0.9687342343460233]
Validation Accuracy = [0.9579999935626984]


In [22]:
history10 = model.fit_generator(training_generator,
                       steps_per_epoch = 100, 
                       epochs = 10,
                       validation_data = validation_generator,
                       validation_steps = 50) 

Epoch 1/10
100/100 [==============================] - 74s 735ms/step - loss: 0.0354 - acc: 0.9890 - val_loss: 0.0515 - val_acc: 0.9840
Epoch 2/10
100/100 [==============================] - 68s 677ms/step - loss: 0.0254 - acc: 0.9940 - val_loss: 0.0398 - val_acc: 0.9830
Epoch 3/10
100/100 [==============================] - 67s 674ms/step - loss: 0.0089 - acc: 0.9995 - val_loss: 0.0493 - val_acc: 0.9830
Epoch 4/10
100/100 [==============================] - 69s 690ms/step - loss: 0.0160 - acc: 0.9960 - val_loss: 0.0283 - val_acc: 0.9890
Epoch 5/10
100/100 [==============================] - 69s 695ms/step - loss: 0.0067 - acc: 0.9995 - val_loss: 0.0440 - val_acc: 0.9810
Epoch 6/10
100/100 [==============================] - 70s 699ms/step - loss: 0.0063 - acc: 0.9990 - val_loss: 0.0496 - val_acc: 0.9850
Epoch 7/10
100/100 [==============================] - 70s 697ms/step - loss: 0.0046 - acc: 0.9995 - val_loss: 0.0332 - val_acc: 0.9900
Epoch 8/10
100/100 [==============================] - 7

In [0]:
from keras.models import load_model

model.save_weights('/content/drive/My Drive/Colab Notebooks/history10.h5')  # creates a HDF5 file 'my_model.h5'
# del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
# model = load_model('my_model.h5')